In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse

from pyquasar import load_mesh, FemDomain, BemDomain, FetiProblem

# FEM one domain test

In [ ]:
def u(p, n):
  #return p[..., 0] ** 2 - p[..., 1] ** 2
  return 2 * p[..., 0] + 3 * p[..., 1] - 4
def flow(p, n):
  #return 2 * p[..., 0] * n[..., 0] - 2 * p[..., 1] * n[..., 1]
  return 2 * n[..., 0] + 3 * n[..., 1]

domain = next(FemDomain(*data) for data in load_mesh('test.geo'))
dir_vector = u(domain.vertices[:domain.ext_dof_count], 0)

domain.assembly({'dirichlet': flow })
sol = domain.solve_neumann(domain.load_vector)

sol -= domain.kernel[0] * (sol @ domain.kernel[0])/(domain.kernel[0] @ domain.kernel[0])
u_ort = u(domain.vertices, 0) - domain.kernel[0] * (u(domain.vertices, 0) @ domain.kernel[0])/(domain.kernel[0] @ domain.kernel[0])

print(f'Neumann problem error: {np.linalg.norm(sol - u_ort)/np.linalg.norm(u_ort)}')
print(f'Dirichlet problem error: {np.linalg.norm(domain.solve_dirichlet(dir_vector) - domain.load_vector)/np.linalg.norm(domain.load_vector)}')

fig, ax = plt.subplots()
ax.set_aspect('equal')
tpc = ax.tripcolor(domain.vertices[:, 0], domain.vertices[:, 1], u_ort - sol, triangles=domain.elements[0][1], shading='gouraud')
fig.colorbar(tpc)
ax.triplot(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1])

# BEM one domain test

In [ ]:
def u(p, n):
  return 2 * p[..., 0] + 3 * p[..., 1] - 4
def flow(p, n):
  return 2 * n[..., 0] + 3 * n[..., 1]

domain = next(BemDomain(*data) for data in load_mesh('test.geo'))
dir_vector = u(domain.vertices[:domain.ext_dof_count], 0)

domain.assembly({ 'dirichlet' : flow })
domain.decompose()
sol = domain.solve_neumann(domain.load_vector)

sol -= domain.kernel[0] * (sol @ domain.kernel[0])/(domain.kernel[0] @ domain.kernel[0])
u_ort = dir_vector - domain.kernel[0] * (dir_vector @ domain.kernel[0])/(domain.kernel[0] @ domain.kernel[0])

print(f'Neumann problem error: {np.linalg.norm(sol - u_ort)/np.linalg.norm(u_ort)}')
print(f'Dirichlet problem error: {np.linalg.norm(domain.solve_dirichlet(dir_vector) - domain.load_vector)/np.linalg.norm(domain.load_vector)}')

sol = domain.calc_solution(sol)
sol -= np.ones_like(sol) * (sol @ np.ones_like(sol))/(np.ones_like(sol) @ np.ones_like(sol))
u_ort = u(domain.vertices, 0) - np.ones_like(sol) * (u(domain.vertices, 0) @ np.ones_like(sol))/(np.ones_like(sol) @ np.ones_like(sol))

print(f'Error in points: {np.linalg.norm(sol - u_ort)/np.linalg.norm(u_ort)}')

fig, ax = plt.subplots()
ax.set_aspect('equal')
tpc = ax.tripcolor(domain.vertices[:, 0], domain.vertices[:, 1], sol - u_ort, triangles=domain.elements[0][1], shading='gouraud')
fig.colorbar(tpc)
ax.triplot(domain.vertices[:, 0], domain.vertices[:, 1], domain.elements[0][1])

# FETI Simple Tests

In [ ]:
def print_feti_information(problem):
  print(f'Feti problem: domains {len(problem.domains)}, dual size {problem.dual_size}, primal size {problem.primal_size}')
  print(f'Condition number estimation {problem.condition_number_estimate():.2f}')

def print_domains_information(domains):
  fem = [domain for domain in domains if isinstance(domain, FemDomain)]
  print(f'FEM domains {len(fem)}, elements {sum(d.element_count for d in fem)}, dof {sum(d.dof_count for d in fem)}')
  bem = [domain for domain in domains if isinstance(domain, BemDomain)]
  print(f'BEM domains {len(bem)}, elements {sum(d.element_count for d in bem)}, dof {sum(d.dof_count for d in bem)}')

In [ ]:
def simple_test(mesh, dirichlet_name, materials, bem = False):
  domains = [(BemDomain if bem else FemDomain)(*data) for data in mesh]
  print_domains_information(domains)

  problem = FetiProblem(domains)
  problem.assembly(dirichlet_name, materials)
  print_feti_information(problem)
  print()

  u = materials.get(dirichlet_name, lambda p, n: 0)
  problem.add_skeleton_projection(u, { name: { dirichlet_name } for name in materials.keys() })

  problem.decompose()
  for precond, Q in [('Dirichlet', 'Diag'), ('Lumped Dirichlet', 'Diag'),
                     ('Dirichlet', 'I'), ('Lumped Dirichlet', 'I'),
                     ('Dirichlet', 'M'), ('Lumped Dirichlet', 'M'),
                     ('I', 'I')]:
    print(f'Precondition = {precond}, Q = {Q}')
    solutions = problem.solve(precond, Q)

    I = sparse.identity(problem.dual_size).todense()
    A = problem.projectT(problem.operator(I))
    M = problem.project(problem.preconditioner(I, precond))
    def cond(A):
      svd = np.linalg.svd(A)[1]
      return svd.max()/svd.min(where=svd > 1e-10, initial=np.inf)
    cond_est = problem.condition_number_estimate()
    print(f'k(A) = {cond(A):.2f}, k(MA) = {(c := cond(M @ A)):.2f}, k(MA)/est = {c/cond_est:.2f}')

    err_norm = 0
    u_norm = 0
    for domain, sol in zip(problem.domains, solutions):
      err_norm = np.linalg.norm(u(domain.vertices, 0) - sol)
      u_norm = np.linalg.norm(u(domain.vertices, 0))

    print(f'Relative error: {err_norm/u_norm:.2e}')

## FEM Linear test

In [ ]:
def u(p, n):
  return 2 * p[..., 0] + 3 * p[..., 1] - 4
def flow(p, n):
  return 2 * n[..., 0] + 3 * n[..., 1]
def source(p, n):
  return 0

materials = { 'dirichlet': u, 'steel': { 'neumann': flow, 'steel': source }}
simple_test(load_mesh('test.geo', 10, 3), 'dirichlet', materials)

## BEM Linear test

In [ ]:
def u(p, n):
  return 2 * p[..., 0] + 3 * p[..., 1] - 4
def flow(p, n):
  return 2 * n[..., 0] + 3 * n[..., 1]

materials = { 'dirichlet': u, 'steel': { 'neumann': flow, 'steel': 0 }}
simple_test(load_mesh('test.geo', 10, 3), 'dirichlet', materials, bem = True)

## FEM Quadratic test

In [ ]:
def u(p, n):
  return p[..., 0] ** 2
def flow(p, n):
  return 2 * p[..., 0] * n[..., 0]

materials = { 'dirichlet': u, 'steel': { 'neumann': flow, 'steel': -2 }}
simple_test(load_mesh('test.geo', 10, 3), 'dirichlet', materials)

## BEM Quadratic test

In [ ]:
def u(p, n):
  return p[..., 0] ** 2
def flow(p, n):
  return 2 * p[..., 0] * n[..., 0]

materials = { 'dirichlet': u, 'steel': { 'neumann': flow, 'steel': -2 }}
simple_test(load_mesh('test.geo', 10, 3), 'dirichlet', materials, bem = True)